### In this notebook we will learn:

* Getting Data: How to import data from PyCaret repository
* Setting up the sqlite database: To store the model artifacts, it is important to have a persistent database through sqlite.
* Setting up Environment: How to setup an experiment in PyCaret and get started with building classification models
* Create & Compare Models: How to create and compare various models, perform stratified cross validation and evaluate classification metrics
* Predict Model: How to make predictions on new / unseen data

## Loading Data 

### 1.1 Importing packages

In [ ]:
# function
import os
import sys

new_directory = "E:/airflow/airflow"
current_directory = os.getcwd()
scripts_path = os.path.abspath(os.path.join(os.getcwd(), '../scripts'))

def change_directory(current_directory, new_directory,scripts_path):
    # Get the current working directory
    print(f'Current directory: {current_directory}')
    # Define the path to change to
    #new_directory = "E:/airflow/airflow"
    try:
        # Change the current working directory
        os.chdir(new_directory)
        # Verify the change
        current_directory = os.getcwd()
        print(f'Current directory changed to: {current_directory}')
    except FileNotFoundError:
        print(f'Error: The directory "{new_directory}" does not exist.')
    except PermissionError:
        print(f'Error: Permission denied to change to "{new_directory}".')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
    # Add the scripts directory to the Python path
    sys.path.append(scripts_path)

    
change_directory(current_directory, new_directory, scripts_path)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Imported Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.utils import *
from pycaret.classification import *
import mlflow
import subprocess

In [ ]:
# Setting up all directory
root_folder = new_directory
database_path = root_folder+"/database/"
data_directory = root_folder+"/data/raw/"
data_profile_path = root_folder+"/data/profile_report/"
intermediate_data_path = root_folder+"/data/interim/"
final_processed_data_path = root_folder+"/data/processed/"


old_data_directory = root_folder+"/data/raw/"
new_data_directory = root_folder+"/data/new/"
intermediate_path = root_folder+"/data/interim/"

# Database
db_path = root_folder+"/database/"
db_file_name = "feature_store_v01.db"
drfit_db_name = "drift_db_name.db"
date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login']

 ### 1.2 Reading the merged data

In [ ]:
%%time
interim_data = "final_train_data_interim_1729850876.csv" # set the data recieved from the previous notebook
dataset = load_data( [f"{intermediate_data_path}{interim_data}",
                            ]
                         )[0] #since we are only loading single data, we can access it with index 0, since it return multiple dfs in list
dataset.shape

In [ ]:
dataset.head()

 ### 1.3 Splitting the data to seen and unseen

In [ ]:
data_for_model, data_unseen = get_validation_unseen_set(dataset, validation_frac=0.05, sample=True, sample_frac=0.1)
print('Data for Modeling: ' + str(data_for_model.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
data_for_model.head()

In [ ]:
data_for_model.columns

### 2 Setting up the sqlite database

In [ ]:
create_sqlit_connection(database_path,r"mlflow_v01.db")
create_sqlit_connection(database_path,r"feature_store_v01.db")
create_sqlit_connection(database_path,r"drift_db_name.db")

In [ ]:
ml_flow_model_path = root_folder+ "/mlruns/8/d62a87a0240f4cc8a01fe9d61b1e6426/artifacts/models/"
ml_flow_path = root_folder+ "/mlruns/8/d62a87a0240f4cc8a01fe9d61b1e6426"

In [ ]:

mlflow.set_tracking_uri("http://Localhost:6006")

In [ ]:
# do not go ahead unless you execute this step and mlflow is isntalled. 
 
#MAKE mlruns FOLDER on root folder
#run this on terminal where you are on root folder. 
# Makse sure to point the database to correct address. Assuming you have same folder structure you can use this

# mlflow server --backend-store-uri sqlite://///home/charliethomasctg/airflow/database/mlflow_v01.db --default-artifact-root /home/charliethomasctg/airflow/mlruns --port=6006 --host=0.0.0.0


### 3 Setting up Environment: 

The `setup()` function initializes the environment in pycaret and creates the transformation pipeline to prepare the data for modeling and deployment. `setup()`must be called before executing any other function in pycaret. 
* It takes two mandatory parameters: a pandas dataframe and the name of the target column. 
* All other parameters are optional and are used to customize the pre-processing pipeline (we will see them in later tutorials).

When `setup()` is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties. The data type should be inferred correctly but this is not always the case. To account for this, PyCaret displays a table containing the features and their inferred data types after setup() is executed. If all of the data types are correctly identified enter can be pressed to continue or quit can be typed to end the expriment. Ensuring that the data types are correct is of fundamental importance in PyCaret as it automatically performs a few pre-processing tasks which are imperative to any machine learning experiment. These tasks are performed differently for each data type which means it is very important for them to be correctly configured.

In [ ]:
# List of date columns
date_columns = ['registration_init_time', 'transaction_date_min', 'transaction_date_max', 'membership_expire_date_max']

# Convert to datetime
for col in date_columns:
    data_for_model[col] = pd.to_datetime(data_for_model[col])

In [ ]:
# No Pre-Processing 
Baseline_model_exp01 = setup(data = data_for_model, target = 'is_churn', 
                   session_id = 42,fix_imbalance=True,ignore_features=['msno'],
                   date_features=['registration_init_time', 'transaction_date_min', 'transaction_date_max', 'membership_expire_date_max'],
                   n_jobs=-1,use_gpu=True,
                   log_experiment=True,experiment_name='Baseline_model_exp01',
                   log_plots=True, log_data=True,
                   verbose=True,
                   log_profile=False)

In [ ]:
X_train, y_train, X_test, y_test = get_train_test_set_from_setup()
X_train.head()

In [ ]:
pipeline = get_transformation_pipeline_from_setup()
pipeline

### 4 Compare models: 

In [ ]:
best_model = compare_models(fold = 5) 

* Two simple words of code (not even a line) have created over 15 models using 10 fold stratified cross validation and evaluated the 6 most commonly used classification metrics (Accuracy, AUC, Recall, Precision, F1, Kappa). 

* The score grid printed above highlights the highest performing metric for comparison purposes only. The grid by default is sorted using 'Accuracy' (highest to lowest) which can be changed by passing the sort parameter. For example compare_models(sort = 'Recall') will sort the grid by Recall instead of Accuracy. 

* If you want to change the fold parameter from the default value of 10 to a different value then you can use the fold parameter. For example compare_models(fold = 5) will compare all models on 5 fold cross validation. Reducing the number of folds will improve the training time.

In [ ]:
#selecting the best model
lgbm  = create_model('lightgbm', fold = 5) 

In [ ]:
lgbm

### 5 Analyzing the model performance

5.1 Learning Curve

In [ ]:
%matplotlib inline

In [ ]:
plot_model(lgbm, plot = 'learning')

5.2 ROC Curve

In [ ]:
plot_model(lgbm, plot = 'auc')

5.3 Precision-recall Curve

In [ ]:
plot_model(lgbm, plot = 'pr')

5.4 Confusion Matrix

In [ ]:
plot_model(lgbm, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})

5.5 Feature Importance

In [ ]:
#top 10 features
plot_model(lgbm, plot='feature') #feature_all -> to check for all features 

5.6 Prediction class distribution

In [ ]:
plot_model(lgbm, plot='error')

5.7 Model Interpretability

In [ ]:
#pip install shap

In [ ]:
# interpret model
interpret_model(lgbm)

In [ ]:
interpret_model(lgbm,plot='correlation',feature='is_cancel')

In [ ]:
interpret_model(lgbm,plot='reason',observation=0) # index of observation in test data

In [ ]:
#pip install interpret

In [ ]:
interpret_model(lgbm,plot='msa')

5.8 Model Evaluation

In [ ]:
# Convert to datetime
for col in date_columns:
    data_unseen[col] = pd.to_datetime(data_unseen[col])
predict_model(lgbm, data_unseen)